In [ ]:
!pip install qdrant-client

In [2]:
# Connect to Qdrant

from qdrant_client import QdrantClient

client = QdrantClient(":memory:")               # in memory and no docker usage

## Experiment 1

a small, simple example using 3-dimensional vectors so you can see similarity search in action

In [3]:
# create collection
# collection is like a 'table' that stores the vectors with metadata

from qdrant_client.models import VectorParams

client.recreate_collection(
    collection_name="first_collection",
    vectors_config=VectorParams(size=3, distance="Cosine")
)

print("collection created!")

collection created!


/tmp/ipython-input-4262839707.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [20]:
# qdrant only able to insert data as an object > use PointStruct (id, vector, metadata)

from qdrant_client.models import PointStruct


vectors = [
    [0.1, 0.2, 0.3],
    [0.5, 0.4, 0.3],
    [0.9, 0.1, 0.2],
    [0.4, 0.5, 0.6],
    [0.7, 0.8, 0.9]
]

payloads = [
    {"name": "A"},
    {"name": "B"},
    {"name": "C"},
    {"name": "D"},
    {"name": "E"}
]


# Insert vectors
client.upsert(
    collection_name="first_collection",
    points=[ PointStruct(id = i, vector = vec, payload = payloads[i]) for i, vec in enumerate(vectors)
           ]
)

print("Vectors inserted!")

Vectors inserted!


In [35]:
# Perform a Similarity Search

query_vector = [0.5, 0.5, 0.5]

result = client.query_points(
    collection_name="first_collection",
    query = query_vector,
    limit=1
)

for r in result:
  print(r)

('points', [ScoredPoint(id=4, version=0, score=0.9948319773377511, payload={'name': 'E'}, vector=None, shard_key=None, order_value=None)])


In [ ]:
# note

# Vectors can be 768 / 1024 / 4096 dimensions

# Returning them every time is:

      # slow
      # memory-heavy
      # usually unnecessary

# Most vector DBs are designed to return IDs + scores (+ optional metadata),
# not raw vectors, unless you explicitly ask.